In [1]:
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow
from tensorflow.keras.datasets import mnist

In [2]:
def get_mnist_model():                      # for the model to be reusable later
  inputs = keras.Input((28 * 28,))
  features = layers.Dense(512, activation='relu')(inputs)
  features = layers.Dropout(0.5)(features)                 # randomly reducing the neurons to reduce overfitting.
  outputs = layers.Dense(10, activation='softmax')(features)
  model = keras.Model(inputs, outputs)
  return model

In [3]:
# Getting the mnist data
(images, labels), (test_images, test_labels) = mnist.load_data()
images = images.reshape((60000, 28 * 28)).astype("float32") / 255
test_images = test_images.reshape((10000, 28 * 28)).astype("float32") / 255
train_images, val_images = images[10000:], images[:10000]
train_labels, val_labels = labels[10000:], labels[:10000]

11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


When writing a training loop, remember to pass "training=True" during the forward pass so that it behaves in a training mode (and Not inference mode)

Also, to retrieve the gradients of the weights of the model, "model.trainable_weights" should be used, so that the weights will be updated via backpropagation to minize the loss of the model, such as in kernel, 'W' and bias, 'b'.

## Low level usage of metrics

in the low-level training loop, you'll likely want to leverage the keras metrics.

for the API: simply call update_state(y_true, y_pred) for each batch and predictions, and then use the result() argument to querry the current metric value. Lets see:

In [4]:
metric = keras.metrics.SparseCategoricalAccuracy()
targets = [0, 1, 2]
predictions = [[1, 0, 0], [0, 1, 0], [0, 0, 1]]
metric.update_state(targets, predictions)
current_result = metric.result()
print(f"result : {current_result:.2f}")

result : 1.00


Also, to track the average of a scaler like the model's loss, you can use the keras.mmetrics.Mean() metric as follows

In [5]:
values = [0, 1, 2, 3, 4]
mean_tracker = keras.metrics.Mean()
for val in values:
  mean_tracker.update_state(val)
print(f"Mean of values : {mean_tracker.result():.2f}")

Mean of values : 2.00


 Remember to use metric.reset_state() when you want to reset the current results (at the start of a training epoch or at the start of evaluation). Just as the one used in the custom metric: "RootMeanSquaredError()" (check Testing.ipynb file for reference)

with all that we've seen, Lets now look at a complete training and evaluation loop.
## Complete training and evaluation loop

In [6]:
# writing the training loop function
model = get_mnist_model()
loss_fn = keras.losses.SparseCategoricalCrossentropy()
optimizer = keras.optimizers.RMSprop()
metrics = [keras.metrics.SparseCategoricalAccuracy()]   # prepares the list of metric to monitor. reports on how well is the model doin
loss_fn = keras.losses.SparseCategoricalCrossentropy()     # keeps track of avrage of the losses

